# Assignment - Solve advection equation numerically
## Exact solution
$u(x,t) = u_0(x-vt)$
## Initial profile
$x\:\epsilon <0,2>$:

pro $x < 0.5$ a $x > 1$: $y = 0$

pro $x > 0.5$ a $x < 1$: $y = 1$

$v = 0.2$

$t_{end} = 2.5$

$N = 100$

$dt = 0.01$

## Boundary conditions
$u_1 = u_n$

## Step by step manual
 * Discretisation $u(x,t)$
 * Initial profile
 * FDT (Forward Difference in Time), BDS (Backward Difference in Space)

In [1]:
import numpy as np
from matplotlib import pyplot as plt


## Discretisation, initial values and initial profile

In [2]:
v = 0.2
t_end = 2.5
dt = 0.01
t_kroku = t_end / dt
N = 100

#Spatial coordinate discretisation
x,dx = np.linspace(0,2,N,retstep=True)

def box_profile(x):
    #Initial profile
    u = np.ones_like(x)

    x_mask = np.where([(el > 0.5) and (el < 1) for el in x])
    u[x_mask] = 2.0
    #print(u)

    return u

u = box_profile(x)


# Exact solution - implementation

In [5]:
# Calculate exact solution of linear-advection equation - copy from Viktor
# for given advection speed v and time t
def solution_box(t,v,u,x):
    u_new = np.empty(np.size(u))
    u_new[np.where((x - v*t) < 0)] = 1.0
    indexes = np.where((x - v*t) > 0)
    u_new[indexes] = box_profile(x[indexes]-v*t)
    return u_new

solution_EX = solution_box(t_end,v,u,x)
#print(solution_EX)
#print(u)


[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  2.  2.  2.  2.
  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
  2.  2.  2.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


# Explicit solution
## FDT - Forward Difference in Time and BDS - Backward Difference in Space - UPWIND

In [10]:
def FDT_BDS(u,x):
    t = 0.0
    u_new = np.ones_like(u)

    while (t < t_end):
        #Forward time and backward space difference
        u_new[1:] = u[1:] - v*(dt/dx)*(u[1:]-u[:-1])

        #Boundary conditions
        u_new[0] = u_new[-1]
        u_new[1] = u_new[-2]

        u = u_new

        #Saving u_new - TO DO
        #Adding timestep
        t += dt
    return u

solution_FB = FDT_BDS(u,x)
print(solution_FB)


[ 1.00000141  1.00000323  1.00000141  1.0000006   1.00000025  1.0000001
  1.00000004  1.00000002  1.00000001  1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.          1.
  1.          1.00000002  1.00000012  1.00000067  1.00000316  1.00001273
  1.0000448   1.00013995  1.00039296  1.00100204  1.00234052  1.00504435
  1.01009488  1.01886297  1.0330734   1.05465763  1.08548883  1.12703233
  1.179983    1.2439822   1.3174996   1.39792777  1.48188192  1.56564226
  1.64564114  1.71889206  1.78328131  1.8376852   1.88192035  1.91656871
  1.94273475  1.96178967  1.97514153  1.9840492   1.98947945  1.99199697
  1.99167918  1.98806051  1.98012628  1.96638736  1.94506218  1.91436933
  1.87289769  1.81998334  1.75600201  1.68249317  1.60206901  1.51811667
  1.43435714  1.35435861  1.28110783  1.21671863  1.16231466  1.11807898
  1.08342813  1.05725252  1.03816552  1.

## FDT - Forward Difference in Time and BDS - Forward Difference in Space - DOWNWIND

In [9]:
def FDT_FDS(u,x):
    t = 0.0
    u_new = np.ones_like(u)

    while (t < t_end):
        #Boundary conditions
        u_new[0] = u_new[-1]
        u_new[1] = u_new[-2]
        
        #Forward time and forward space difference
        u_new[:-1] = u[:-1] - v*(dt/dx)*(u[1:]-u[:-1])
        u = u_new
        #Saving u_new - TO DO
        #Adding timestep
        t += dt
    return u

solution_FF = FDT_FDS(u,x)
print(solution_FF)


[  1.00000000e+00   1.64658805e+17  -2.00811306e+18   2.18836453e+18
  -2.26761045e+18   2.22929532e+18  -2.07420119e+18   1.82153893e+18
  -1.50530069e+18   1.16671814e+18  -8.45139829e+17   5.70237449e+17
  -3.57855138e+17   2.10303361e+17  -1.20209512e+17   7.59573268e+16
  -6.66665634e+16   8.54028369e+16  -1.30306638e+17   2.04046016e+17
  -3.12205796e+17   4.61064671e+17  -6.54988686e+17   8.93629678e+17
  -1.16931152e+18   1.46528531e+18  -1.75571533e+18   2.00811211e+18
  -2.18836207e+18   2.26760423e+18  -2.22928003e+18   2.07416459e+18
  -1.82145370e+18   1.50510769e+18  -1.16629341e+18   8.44232052e+17
  -5.68354240e+17   3.54065448e+17  -2.02910592e+17   1.06239240e+17
  -5.04023762e+16   2.14521761e+16  -8.09071688e+15   2.66193734e+15
  -7.48500464e+14   1.74897206e+14  -3.26057665e+13   4.54733988e+12
  -4.21780716e+11   1.95173477e+10   1.00000000e+00   1.00000000e+00
   1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
   1.00000000e+00   1.00000000e+00

## FDT - Forward Difference in Time and BDS - Central Difference in Space

In [8]:
def FDT_CDS(u,x):
    t = 0.0
    u_new = np.ones_like(u)

    while (t < t_end):
        #Boundary conditions
        u_new[0] = u_new[-1]
        u_new[1] = u_new[-2]
        
        #Forward time and forward space difference
        u_new[1:-1] = u[1:-1] - v*(dt/(2*dx))*(u[2:]-u[:-2])
        u = u_new
        #Saving u_new - TO DO
        #Adding timestep
        t += dt
    return u

solution_FC = FDT_CDS(u,x)
print(solution_FC)


[ 1.          1.01135013  0.76572236  1.09636259  0.86191693  0.9951877
  1.09856175  0.80857031  1.15876946  0.9882301   0.82841501  1.22674612
  0.93276719  0.81444472  1.25743535  0.9781654   0.73158826  1.21638226
  1.15712212  0.68110714  0.97232461  1.361031    0.90679567  0.67390728
  1.06968123  1.45391941  0.76622903  0.75482455  0.88452479  1.51822342
  1.10270218  0.52929409  0.88199924  1.01838941  1.43636512  1.31728885
  0.47613598  0.65182812  1.0642275   1.01938147  1.45895515  1.53384364
  0.70143006  0.46523988  0.84874324  0.67056477  0.55140276  1.27986521
  1.83058764  1.55553593  1.45065549  2.01762363  2.34686215  1.94293479
  1.57878054  1.88126278  2.36215924  2.3049045   1.82089444  1.57460836
  1.85398536  2.29700256  2.415362    2.12131807  1.72316521  1.56291666
  1.73446032  2.08247204  2.3769197   2.47187207  2.35320069  2.09593239
  1.79655128  1.5268923   1.32057297  1.1808941   1.095297    1.04712194
  1.02196796  1.00969189  1.0040595   1.00161877  1.